# Introduction

In [1]:
NAME = '1. obtain and clean data' 
PROJECT = 'before trilogy'
PYTHON_VERSION = '3.6'

# Preamble

### Imports  

In [2]:
import os, re, math, time
import requests, zipfile, io, os
from tqdm.notebook import tqdm

In [3]:
import requests
import requests_html
import lxml.html

In [4]:
import pandas as pd 
import numpy as np

In [5]:
import spacy
import en_core_web_md
nlp = en_core_web_md.load()

In [6]:
import nltk
from nltk.corpus import words

### Set working directory  

In [7]:
workdir = re.sub("(?<={})[\w\W]*".format(PROJECT), "", os.getcwd())
os.chdir(workdir)

--- 
# Main code

## 1. Get the scripts

### 1.1. Mignight

The script is found:
https://www.scripts.com/script.php?id=before_midnight_51&p=1

In [8]:
# create a session object for requests
session = requests_html.HTMLSession()

In [9]:
# Get the strips from all pages (66 papes in total)
midnight_script_list = []

for page_number in range(1, 67):
    # load the source from the URL + parse HTML
    endpoint = 'https://www.scripts.com/script.php?/{}'
    params = {'id': 'before_midnight_51', 'p':page_number}
    res = session.get(endpoint, params = params)
    
    # extract information from the HTML structure
    script_elements = res.html.find('#disp-quote-body')

    # remove the part that is not script
    text_elements = script_elements[0].text.split('\nRate this script')[0]
    
    midnight_script_list.append(text_elements)

In [10]:
midnight_script_full = ' '.join([x for x in midnight_script_list]) # remove all the duplicated whitespace

In [11]:
# find the patterns in the text that can identify who says what.
midnight_dial_list = re.findall('([A-Z\s]+):(.*?)(?=[A-Z\s]+:|$)', midnight_script_full, flags=re.DOTALL)
# () matches groups, and will return as the element in tuple
# [A-Z\s]+ captures capital letters (since the script will capital letters to indicate talking person)
# \s: matches any white spaces characters (e.g., space, tabs, line breakers)
# .*? captures everything
# ?= captures a group after the main expression without including it in the result.
# |$ makes sure that we do not miss the last sentence

In [12]:
# clean the text
def clean_text(string):
    ## Remove cont'd thing
    string = re.sub("([A-Z\s]+\s+\(CONT'D\))", ' ', string)
    
    ## Remove the explaining action
    string = re.sub("(\(.*?\))", ' ', string)
    
    ## Clean rest
    string = string.replace('\n', ' ')
    string = ' '.join([x for x in string.split() if x]) # remove all the duplicated whitespace
    string = string.strip()
    
    return string

In [13]:
midnight_dialogue_list = []
for speaker, text in midnight_dial_list:
    ret_dict = {'speaker' : clean_text(speaker), 'text' : clean_text(text)}
    midnight_dialogue_list.append(ret_dict)
# the data structure here is a list of dictionaries.
# here, we have a speaker who is assoicated with multiple texts. Thus, we cannot use speaker name as the key of the dictionary.

In [14]:
midnight_dialogue_df = pd.DataFrame(midnight_dialogue_list)

In [15]:
midnight_dialogue_df.to_csv('0_data\midnight_rawscript.csv', index=False, encoding='utf-8')

### 1.2. Before sunrise

The script is found: 
https://sunrisesunset.fandom.com/wiki/Before_Sunrise_script

In [16]:
# create a session object for requests
session = requests_html.HTMLSession()

In [17]:
# load the source from the URL + parse HTML
res = session.get(r'https://sunrisesunset.fandom.com/wiki/Before_Sunrise_script')

In [18]:
script_elements = res.html.find('p')
# another option is '.mw-parser-output'

In [19]:
len(script_elements)
# it turns out the first one captures all the text we want. 

971

In [20]:
# remove the heading
text_elements = script_elements[0].text.split('\nauf?!\n\n')[1]
# remove the ending
sunrise_script_full = text_elements.split('\n\n\n(They kiss again')[0]

In [21]:
# replace Céline to celine, otherwise, it will not work with regexp
sunrise_script_full = re.sub('Céline', 'Celine', sunrise_script_full)

In [22]:
# find the patterns in the text that can identify who says what.
sunrise_dial_list = re.findall('([A-Z][a-z\s]+):(.*?)(?=[A-Z][a-z\s]+:|$)', sunrise_script_full, flags=re.DOTALL)

In [23]:
# use the clean_text function that we define above to clean text data
sunrise_dialogue_list = []
for speaker, text in sunrise_dial_list:
    ret_dict = {'speaker' : clean_text(speaker), 'text' : clean_text(text)}
    sunrise_dialogue_list.append(ret_dict)

In [24]:
sunrise_dialogue_df = pd.DataFrame(sunrise_dialogue_list)

In [25]:
sunrise_dialogue_df.to_excel('0_data\sunrise_rawscript.xlsx', index=False)

### 1.3. Before sunset

The script is found:
https://sunrisesunset.fandom.com/wiki/Before_Sunset_(2004)_script

In [26]:
# create a session object for requests
session = requests_html.HTMLSession()

In [27]:
# load the source from the URL + parse HTML
res = session.get(r'https://sunrisesunset.fandom.com/wiki/Before_Sunset_(2004)_script')

In [28]:
script_elements = res.html.find('p')
# another option is '.mw-parser-output'

In [29]:
len(script_elements)
# it turns out the first one captures all the text we want. 

946

In [30]:
# remove the heading
text_elements = script_elements[0].text.split('The bookstore owner sits to Jesse’s right.)\n')[1]
# remove the ending
sunset_script_full = text_elements.split('\n\n\n(Céline continues her dance as the scene fades to black.)')[0]

In [31]:
# replace Céline to celine, otherwise, it will not work with regexp
sunset_script_full = re.sub('Céline', 'Celine', sunset_script_full)

In [32]:
# find the patterns in the text that can identify who says what.
sunset_dial_list = re.findall('([A-Z][a-z\s]+):(.*?)(?=[A-Z][a-z\s]+:|$)', sunset_script_full, flags=re.DOTALL)

In [33]:
# use the clean_text function that we define above to clean text data
sunset_dialogue_list = []
for speaker, text in sunset_dial_list:
    ret_dict = {'speaker' : clean_text(speaker), 'text' : clean_text(text)}
    sunset_dialogue_list.append(ret_dict)

In [34]:
sunset_dialogue_df = pd.DataFrame(sunset_dialogue_list)

In [35]:
sunset_dialogue_df.to_excel('0_data\sunset_rawscript.xlsx', index=False)

## 2. Clean data

### 2.1. Midnight

In [36]:
midnight = pd.read_csv('0_data\midnight_addscene.csv', encoding_errors='ignore')

In [37]:
# clean incorrect speaker names
midnight.speaker = midnight.speaker.replace('JEESE', 'JESSE', regex=False)
midnight.speaker = midnight.speaker.replace('JESE', 'JESSE', regex=False)
midnight.speaker = midnight.speaker.replace('AFTERNOON JESSE', 'JESSE', regex=False)

In [38]:
# delete lines that are not speakers
midnight = midnight[(midnight.speaker != 'FADE IN') & (midnight.speaker != 'FADE OUT')]

In [39]:
# check if there is null columns in the text, otherwise, it will throw errors later
# midnight[midnight.isnull().any(axis=1)] 

# It turns out some rows are empty. delete them. 
midnight = midnight.dropna()

In [40]:
midnight.speaker.value_counts()

JESSE             533
CELINE            490
STEFANOS           64
ARIADNI            35
HANK               30
ACHILLES           21
ANNA               21
PATRICK            17
SOFIA              12
ELLA                4
NATALIA             3
GROUP               3
HOTEL CLERK         2
NINA                1
OTHERS              1
PATRICK/CELINE      1
Name: speaker, dtype: int64

In [41]:
# lower case the speaker and text
midnight.text = midnight.text.str.lower()
midnight.speaker = midnight.speaker.str.lower()
midnight.scene_name = midnight.scene_name.str.lower()

In [42]:
midnight.to_excel('2_pipeline\midnight_for_analysis.xlsx', index=False)

### 2.2. Sunrise

In [43]:
sunrise = pd.read_excel('0_data\sunrise_addscene.xlsx')

In [44]:
# drop lines that are associated with non-speakers
sunrise = sunrise[(sunrise.speaker != 'Location notes') 
                  & (sunrise.speaker != 'Scene clip') 
                  & (sunrise.speaker != 'Web') 
                  & (sunrise.speaker != 'Bus')
                  & (sunrise.speaker != 'Metro stop')
                  & (sunrise.speaker != 'Metro')
                  & (sunrise.speaker != 'Museum is at http')
                  & (sunrise.speaker != 'Scene trivia')
                  & (sunrise.speaker != 'Note')
                  & (sunrise.speaker != 'Tram')
                  & (sunrise.speaker != 'Public transit')
                  & (sunrise.speaker != 'Web')]

In [45]:
# check if there is null columns in the text, otherwise, it will throw errors later
# sunrise[sunrise.isnull().any(axis=1)] 
# It turns out two rows are empty. delete them. 
sunrise = sunrise.dropna()

In [46]:
sunrise.speaker.value_counts()

Jesse              356
Celine             345
Man with tie        21
Poet                11
Man with jacket     10
Gypsy                6
Bartender            6
Man                  4
Woman                2
Man on left          2
music                1
Doorman              1
Musician             1
Man on right         1
Man right            1
Older man            1
Name: speaker, dtype: int64

In [47]:
# lower case the speaker and text
sunrise.text = sunrise.text.str.lower()
sunrise.speaker = sunrise.speaker.str.lower()
sunrise.scene_name = sunrise.scene_name.str.lower()

In [48]:
sunrise.to_excel('2_pipeline\sunrise_for_analysis.xlsx', index=False)

### 2.3 Sunset

In [49]:
sunset = pd.read_excel('0_data\sunset_addscene.xlsx')

In [50]:
# drop lines that are associated with non-speakers
sunset = sunset[(sunset.speaker != 'Location notes')
                & (sunset.speaker != 'Metro')]

In [51]:
# check if there is null columns in the text, otherwise, it will throw errors later
#sunset[sunset.isnull().any(axis=1)]  
sunset = sunset.dropna()

In [52]:
sunset.speaker.value_counts()

Jesse               381
Celine              376
Bookstore owner       6
Female reporter       5
Attendant             3
Philippe              3
Man at the grill      3
Woman                 3
Male reporter         2
Waitress              1
Man                   1
Name: speaker, dtype: int64

In [53]:
# lower case the speaker and text
sunset.text = sunset.text.str.lower()
sunset.speaker = sunset.speaker.str.lower()
sunset.scene_name = sunset.scene_name.str.lower()

In [54]:
sunset.to_excel('2_pipeline\sunset_for_analysis.xlsx', index=False)